---
author: Dazhong Li
date: 4 Dec 2017
...

# Introduction 

This workbook generate a material table from the Plaxis output for an easier check of the Plaxis material input.

# Output from Plaxis

The python code works on the plaxis output geneated with the Plaxis material output.

Following steps should be followed in order to generate a material table:
 
 - In the Plaxis Output, goto <file>
 - Selecte the <Report generater>
 - Selecte the <Seperate files>
 - Selected the stage you are intersted
 - Selecte the material information
 - Leave everytin else a default, seperate files will be generated, noted the path of your files, by default it should be under the "Report" folder in your working directory, feed the path to the path variable below

In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option('precision', 2)
pd.set_option('display.width', 1000)

def construct_formula(phi,referenc_level, c_ref,c_inc):
    if c_inc !=0:
        try:
            return "Cu = {:.2f} + {:.2f}z \n z=0 at {:.1f}mPD".format(c_ref,c_inc,referenc_level)
        except:
            return ''
    else:
        return 'c = {:.1f} ; phi = {:.0f}'.format(c_ref,phi)

Input the package of the report generated from Plaxis output, note sometimes, with display width of Pandas need to be adjusted to get the better output

In [2]:
def generate_soil_strength_table(path):
    os.chdir(path)
    list_ = []
    soil_frame = pd.DataFrame()
    for subdir, dirs, files in os.walk('./'):
        count = 0 
        for file in files:  
            if 'Soil and interfaces' in file:
                df = pd.read_csv(os.path.join(path,file),sep='\t')
                if count>0:
                    df.drop(df.columns[[0, 1]], axis=1, inplace = True)
                list_.append(df)
                count = count + 1
    soil_frame = pd.concat(list_,axis=1)  
    strength_parameter_index = [0,1,4,17,23,16,24,12,22,13]   
    frame_strength = soil_frame.iloc[strength_parameter_index,:]

    #soil_frame.to_csv('output.csv')
    #frame_strength.sort_index(axis = 1,inplace = True)
    #frame_strength = frame_strength.transpose().to_csv('Strength.csv')
    #print(soil_frame)
    frame_strength= frame_strength.transpose()
    frame_strength.columns=['Material ID','Drainage','Gamma','Phi','y_ref','c_ref','c_inc','E','E_inc','nu']
    cols = frame_strength.columns[list(np.arange(2,9))]
    frame_strength[cols] = frame_strength[cols].apply(pd.to_numeric, errors='coerce', axis=1)
    frame_strength.drop(frame_strength.index[[0,1]],inplace=True)
    frame_strength.drop(frame_strength.index[[ '#' in x for x in frame_strength.index]],inplace=True)
    try:
        frame_strength.drop('Void',inplace=True)
    except:
        pass
    frame_strength.sort_index(inplace=True)
    frame_strength['formula'] = frame_strength.apply(lambda row: construct_formula(row['Phi'],
                                                                                   row['y_ref'],
                                                                                   row['c_ref'],
                                                                                   row['c_inc']),axis=1)
    frame_strength['Material ID'] = frame_strength['Material ID'].apply(lambda x: '{}'.format(x).rjust(2,'0'))
    
    return frame_strength

In [3]:
path  = r'C:\Users\dazhong.li\Desktop\report'

frame_strength = generate_soil_strength_table(path)
frame_strength = frame_strength.sort_values(by = 'Material ID')
frame_strength.to_excel('soil properties.xlsx')

In [4]:
frame_strength.sort_values(by='Material ID').to_html('soil_parameters.html')

In [5]:
frame_strength = frame_strength.sort_values(by = 'Material ID')

In [6]:
frame_strength

,Material ID,Drainage,Gamma,Phi,y_ref,c_ref,c_inc,E,E_inc,nu,formula
Sand/Public Fill (Uncompacted),01,Drained,19.0,30.0,0.0,0.05,0.00,10000.0,0.0,0.3300,c = 0.1 ; phi = 30
Hydraulic Sand Fill,02,Drained,19.0,30.0,0.0,0.05,0.00,10000.0,0.0,0.3300,c = 0.1 ; phi = 30
MD - Udr(B) (In Situ),03,Undrained (B),15.0,0.0,-5.0,1.20,1.05,135.0,35.0,0.2500,Cu = 1.20 + 1.05z \n z=0 at -5.0mPD
Alluvium - UDr,04,Drained,19.0,0.0,0.0,60.00,0.00,14400.0,0.0,0.4950,c = 60.0 ; phi = 0
CDG,05,Drained,19.0,34.0,-50.0,0.10,0.00,58000.0,1000.0,0.2500,c = 0.1 ; phi = 34
Alluvial Clay/Silt/Sand,07,Drained,19.0,28.0,-20.0,2.50,0.00,10120.0,874.0,0.2500,c = 2.5 ; phi = 28
Rock Fill (Berm),08,Drained,22.0,45.0,0.0,0.05,0.00,40000.0,0.0,0.3000,c = 0.1 ; phi = 45
Alluvium - UDr [125kPa],09,Drained,19.0,0.0,0.0,125.00,0.00,31250.0,0.0,0.4950,c = 125.0 ; phi = 0
Sand Fill (Compacted),10,Drained,19.0,35.0,0.0,0.05,0.00,15000.0,0.0,0.3000,c = 0.1 ; phi = 35
Alluvium - UDr [100kPa],11,Drained,19.0,0.0,0.0,100.00,0.00,25000.0,0.0,0.4950,c = 100.0 ; phi = 0
